<p style="font-weight:bold;"> <span style="font-size: 24px"> <span style="color:#0060E2">Systemorph Cloud</span> </span> </p>
<br/><br/>
<p style="font-weight:bold;"> <span style="font-size: 36px"> <span style="color:#009A50"> Chess Board </span> </p>
<br/><br/>
<b>Authors:</b> Andrea Muolo, Nikos Nikolopoulos
<br/><br/><br/>

In [0]:
#r "nuget:Systemorph.Activities,1.6.5"
#r "nuget:Systemorph.Workspace,1.6.4"
#r "nuget:Systemorph.Reporting,1.6.5"
#r "nuget:Systemorph.Export,1.6.7"
#r "nuget:Systemorph.Import,1.6.7"
#r "nuget:Systemorph.Scopes,1.6.5"
#r "nuget:Systemorph.InteractiveObjects,1.6.5"

In [0]:
using Microsoft.Extensions.Logging;
using Systemorph.Vertex.Activities;
using Systemorph.Vertex.Workspace;
using Systemorph.Vertex.Pivot.Builder.Interfaces;
using Systemorph.Vertex.Export.Factory;
using Systemorph.Vertex.Export;
using Systemorph.Vertex.Import;
using Systemorph.Vertex.Grid.Model;

# Enum Declaration

## Colors

In [0]:
public enum Colors {
    Black, White, NoColor
}

## Messages

In [0]:
public enum Messages {
    NextPlayerColor, WinnerMessage
}

## Errors

In [0]:
public enum Errors {
    InvalidLength, InvalidInput, EmptySelection, WrongColor, IllegalMove, InvalidForm
}

## Pieces

In [0]:
public enum Pieces {
    Pawn, Rook, Knight, Bishop, Queen, King, NoPiece
}

# Utilities

## Mappings

In [0]:
public static class Mappings
{    
    public static int ToNum (string x) {
        switch (x) {
            case "A" : case "1" : return 1; case "B" : case "2" : return 2;
            case "C" : case "3" : return 3; case "D" : case "4" : return 4; 
            case "E" : case "5" : return 5; case "F" : case "6" : return 6;
            case "G" : case "7" : return 7; case "H" : case "8" : return 8;
 
            default : return 0;
        }
    }
        
    public static string ToLet (int x) {
        switch (x) {
            case 1 : return "A"; case 2 : return "B";
            case 3 : return "C"; case 4 : return "D"; 
            case 5 : return "E"; case 6 : return "F";
            case 7 : return "G"; case 8 : return "H";

            default : return "Z";
        }
    }
}

## Application Message

In [0]:
public static string GetMessage(Messages message, Colors blackOrWhite = Colors.NoColor)
{
    switch (message) {
        case Messages.NextPlayerColor: return $"\n{blackOrWhite}'s turn!";
        case Messages.WinnerMessage: return $"Congratulations!!! {blackOrWhite}'s win!";

        default: return null;
    }
}

In [0]:
public static string GetMessage(Errors error, Colors blackOrWhite = Colors.NoColor)
{
    switch (error) {
        case Errors.InvalidLength:  return "Input should be 2 characters long.";
        case Errors.InvalidForm:    return "Input has invalid form.";
        case Errors.InvalidInput:   return "Input is invalid.";
        case Errors.WrongColor:     return $"Select a {blackOrWhite} piece.";
        case Errors.EmptySelection: return "The cell you have selected is empty.";
        case Errors.IllegalMove:    return "Illegal operation! Please try again.";

        default:                    return "Error not found.";
    }
}

## Get Symbol

In [0]:
public static string GetSymbol(Pieces kind, Colors color)
{
    switch (kind, color) {
        case (Pieces.Pawn,    Colors.White):   return "♙";  case (Pieces.Pawn,      Colors.Black):   return "♟";
        case (Pieces.Knight,  Colors.White):   return "♘";  case (Pieces.Knight,    Colors.Black):   return "♞";
        case (Pieces.Rook,    Colors.White):   return "♖";  case (Pieces.Rook,      Colors.Black):   return "♜";
        case (Pieces.Bishop,  Colors.White):   return "♗";  case (Pieces.Bishop,    Colors.Black):   return "♝";
        case (Pieces.Queen,   Colors.White):   return "♕";  case (Pieces.Queen,     Colors.Black):   return "♛";
        case (Pieces.King,    Colors.White):   return "♔";  case (Pieces.King,      Colors.Black):   return "♚";
        
        default: return null;
    }
}

# Data Types

## Coordinate

In [0]:
public record Coordinate
{
    public string Letter {get; init;}
    public int Number    {get; init;}
    
    public static ILogger log;

    public Coordinate (string letter, int number) {
        Letter = letter;
        Number = number;
        if(!this.IsValid())
            log.LogError(GetMessage(Errors.InvalidInput));
    }

    public Coordinate (int letter, int number) {
        Letter = Mappings.ToLet(letter);
        Number = number;
        if(!this.IsValid())
            log.LogError(GetMessage(Errors.InvalidInput));
    }
    
    public bool IsValid() => ( Mappings.ToNum(Letter) >= 1 && 
                               Mappings.ToNum(Letter) <= 8 && 
                               Number >= 1 && Number <= 8 );
}

In [0]:
Coordinate.log = Log;

## Import Export

In [0]:
public record ImportExport 
{
    [NotVisible] public string Letter { get; init; }
    [NotVisible] public int Number    { get; init; }
    [NotVisible] public Pieces Kind   { get; init; }
    [NotVisible] public Colors Color  { get; init; }

    public string Symbol => GetSymbol(Kind, Color);
}

## Piece

In [0]:
public record Piece (Colors Color, Pieces Kind);

# Checks

## Is Piece in Path

In [0]:
public static bool IsPieceInPath(this Dictionary<Coordinate, Piece> pieceByCoordinate, Coordinate origin, Coordinate destination)
{
    var x1 = Mappings.ToNum(origin.Letter);
    var x2 = Mappings.ToNum(destination.Letter);
    var y1 = origin.Number;
    var y2 = destination.Number;

    var deltax = x2 - x1; 
    var deltay = y2 - y1;

    for (var step = 1; step < Math.Max(Math.Abs(deltax), Math.Abs(deltay)); step++) {
        var x = x1 + step * Math.Sign(deltax);
        var y = y1 + step * Math.Sign(deltay); 
        var coordinate = new Coordinate(Mappings.ToLet(x), y);
        if(pieceByCoordinate.ContainsKey(coordinate)) return true;
    }
    return false;
}

## Is Legal Move

In [0]:
public static bool IsLegalMovePiece(this Piece piece, Coordinate origin, Coordinate destination)
{
    int x1 = Mappings.ToNum(origin.Letter);
    int y1 = origin.Number;
    int x2 = Mappings.ToNum(destination.Letter);
    int y2 = destination.Number;

    switch (piece.Kind) {
        case Pieces.Pawn:   return ( (piece.Color == Colors.White && (  y2-y1 == 1 || (y2==4 && y1==2 && x1==x2) ) ) ||
                                     (piece.Color == Colors.Black && (  y1-y2 == 1 || (y2==5 && y1==7 && x1==x2) ) ) );
        case Pieces.Rook:   return ( x2 == x1 || y2 == y1 );
        case Pieces.Knight: return ( (Math.Abs(x1 - x2) == 1 && Math.Abs(y1 - y2) == 2) || 
                                     (Math.Abs(x1 - x2) == 2 && Math.Abs(y1 - y2) == 1) );
        case Pieces.Bishop: return ( Math.Abs(x1 - x2) == Math.Abs(y1 - y2) );
        case Pieces.Queen:  return ( (x2 == x1 || y2 == y1) || (Math.Abs(x1 - x2) == Math.Abs(y1 - y2)) );
        case Pieces.King:   return ( Math.Abs(y2 - y1) == 1 || Math.Abs(x2 - x1) == 1 );
        default: return false;
    }
}

In [0]:
public static bool IsLegalMove(this Dictionary<Coordinate, Piece> pieceByCoordinate, Piece piece, Coordinate origin, Coordinate destination)
{
    bool isPieceInPath = IsPieceInPath(pieceByCoordinate, origin, destination);
    bool isPieceAtDestination = pieceByCoordinate.TryGetValue(destination, out var pieceAtDestination);

    if (isPieceAtDestination && pieceAtDestination.Color == piece.Color) return false;

    int x1 = Mappings.ToNum(origin.Letter);
    int x2 = Mappings.ToNum(destination.Letter);
    var y1 = origin.Number;
    var y2 = destination.Number;

    switch (piece.Kind) {
        case Pieces.Pawn:   return( isPieceAtDestination && Math.Abs(x2-x1) < 2 && Math.Abs(y2-y1) < 2 ) ||
                                  (!isPieceAtDestination && Math.Abs(x2-x1) < 1 ) && 
                                   !isPieceInPath;
        case Pieces.Rook:   return !isPieceInPath;
        case Pieces.Bishop: return !isPieceInPath;
        case Pieces.Queen:  return !isPieceInPath;
        default:            return true;
    }
}

# Scopes

## Context

In [0]:
public interface Context : IMutableScope 
{
    [NotVisible] IActivityVariable  Activity    {get; set;}
    [NotVisible] ILogger            Log         {get; set;}
    [NotVisible] IExportVariable    Export      {get; set;}
    [NotVisible] IImportVariable    Import      {get; set;}
    [NotVisible] IPivotFactory      Report      {get; set;}
    [NotVisible] IWorkspaceVariable Workspace   {get; set;}
}

In [0]:
InteractiveObject.State.GetScope<Context>().Activity = Activity;
InteractiveObject.State.GetScope<Context>().Log = Log;
InteractiveObject.State.GetScope<Context>().Export = Export;
InteractiveObject.State.GetScope<Context>().Import = Import;
InteractiveObject.State.GetScope<Context>().Report = Report;
InteractiveObject.State.GetScope<Context>().Workspace = Workspace;

## Chess Board

In [0]:
[InitializeScope(nameof(InitializeBoard))]
public interface ChessBoard : IMutableScope 
{
    private Context Context => GetScope<Context>();
    List<IActivityNotification> Logs { get; set; }
    Dictionary<Coordinate, Piece> PieceByCoordinate { get; set; }
    [DefaultValue(true)] bool IsWhitesTurn { get; set; }

    void NextMove(string originInput, string destinationInput) {
        Context.Activity.Start();
        var origin = new Coordinate(originInput[0].ToString(), Mappings.ToNum(originInput[1].ToString()));
        if (!PieceByCoordinate.TryGetValue(origin, out var piece)) 
            Context.Log.LogError(GetMessage(Errors.EmptySelection));
        else if ( (IsWhitesTurn && piece.Color == Colors.Black) || 
                 (!IsWhitesTurn && piece.Color == Colors.White) )
            Context.Log.LogError(GetMessage(Errors.WrongColor, IsWhitesTurn ? Colors.White : Colors.Black));

        var destination = new Coordinate( destinationInput[0].ToString(), Mappings.ToNum(destinationInput[1].ToString()) );

        if (piece!=null && !piece.IsLegalMovePiece(origin, destination)) Context.Log.LogError(GetMessage(Errors.IllegalMove));
        if (piece!=null && !IsLegalMove(PieceByCoordinate, piece, origin, destination)) Context.Log.LogError(GetMessage(Errors.IllegalMove));
        
        var errors = Context.Activity.Finish().Errors;
        Logs = errors.ToHashSet().ToList();
        if(errors.Any()) return;

        PieceByCoordinate.Remove(origin);
        PieceByCoordinate.Remove(destination);
        PieceByCoordinate.Add(destination, piece);
        PieceByCoordinate = PieceByCoordinate.ToDictionary(kvp => kvp.Key, kvp => kvp.Value);

        IsWhitesTurn = !IsWhitesTurn;
    }
    
    void InitializeBoard() {
        PieceByCoordinate = new Dictionary<Coordinate, Piece>();
        IsWhitesTurn = true;
        Enumerable.Range(1, 8).ToList().ForEach( column => {
            new[]{1,2,7,8}.ToList().ForEach( row => {
                var color = row < 5? Colors.White : Colors.Black;
                var piece = (row, column) switch {
                    (2 or 7, _) => new Piece(color, Pieces.Pawn),
                    (_, 1 or 8) => new Piece(color, Pieces.Rook),
                    (_, 2 or 7) => new Piece(color, Pieces.Knight),
                    (_, 3 or 6) => new Piece(color, Pieces.Bishop),
                    (_, 4     ) => new Piece(color, Pieces.Queen),
                    (_, 5     ) => new Piece(color, Pieces.King),
                    (_,      _) => null
                };
                PieceByCoordinate[new Coordinate(column, row)] = piece;
            } );
        } );
    }
}

## Input Output

In [0]:
public interface IO : IMutableScope 
{
    [NotVisible] ChessBoard Board => GetScope<ChessBoard>();
    
    void Resume (List<ImportExport> items) {
        Board.PieceByCoordinate.Clear();
        items.ForEach(item => Board.PieceByCoordinate.Add(new Coordinate(item.Letter, item.Number), new Piece(item.Color, item.Kind)) );
    }
    
    List<ImportExport> PiecesList () => Board.PieceByCoordinate.Select(x => 
                                        new ImportExport { Letter = x.Key.Letter, 
                                                           Number = x.Key.Number, 
                                                           Kind = x.Value.Kind, 
                                                           Color = x.Value.Color}).ToList();

    List<ImportExport> PiecesListWithEmptyCells () {
        var items = PiecesList();
        Enumerable.Range(1, 8).ToList().ForEach( i => {
            Enumerable.Range(1, 8).ToList().ForEach( j => {
                var coordinate = new Coordinate(i, j);
                if(!items.Any(x => x.Letter == coordinate.Letter && x.Number == coordinate.Number))
                    items.Add(new ImportExport { Letter = coordinate.Letter, 
                                                 Number = coordinate.Number, 
                                                 Kind = Pieces.NoPiece, 
                                                 Color = Colors.NoColor }); }); });
        return items.ToList();
    }
}

## Application

In [0]:
public interface Application : IMutableScope 
{
    private IO IO => GetScope<IO>();
    private Context Context => GetScope<Context>(); 

    Colors Turn => IO.Board.IsWhitesTurn ? Colors.White : Colors.Black;

    [DropdownValues("","New Game")] [DefaultValue("")] string Controls { 
        get => ""; 
        set { if(value == "New Game") {
                IO.Board.InitializeBoard();
                ResetAllDropDownValues(); } } 
    }

    [DropdownValues("","Play")] [DefaultValue("")] string Action { 
        get => ""; 
        set { if(value == "Play") {
                IO.Board.NextMove(OriginCoordinates, DestinationLetter + DestinationNumber);
                ResetAllDropDownValues(); } }
    }
    
    [DropdownMethod(nameof(GetOriginCoordinatesOptions))] string OriginCoordinates { get; set; }
    [DropdownValues("","A","B","C","D","E","F","G","H")]  [DefaultValue("")] string DestinationLetter { get; set; }
    [DropdownValues("","1","2","3","4","5","6","7","8")]  [DefaultValue("")] string DestinationNumber { get; set; }

    private string ChessBoardStyle => @"function(params) {
            var colsOdd  = ['A.Symbol', 'C.Symbol', 'E.Symbol', 'G.Symbol'];
            var colsEven = ['B.Symbol', 'D.Symbol', 'F.Symbol', 'H.Symbol'];

            var isEvenRow = params.rowIndex % 2 == 0;
            var colId     = params.colDef.colId;

            return { 'background-color': (colsOdd.includes(colId) && isEvenRow) || (colsEven.includes(colId) && !isEvenRow) ? 'grey' : 'initial',
                     'font-weight'     : '500',
                     'font-size'       : '25px',
                     'display'         : 'flex',
                     'align-items'     : 'center',
                     'justify-content' : 'center',
                     'height'          : '100%' };
            }";
    private string ChessCellStyle => @"function() {
        return { 'display'        : 'flex',
                 'text-align'     : 'center',
                 'justify-content': 'center',
                 'height'         : '100%',
                 'font-weight'    : 'bold',
                 'border-right'   : '2px solid lightgrey' };
        }";

    List<string> GetOriginCoordinatesOptions() => IO.Board.PieceByCoordinate.Where(x => x.Value.Color == Turn)
                                                                            .Select(x => x.Key.Letter + x.Key.Number)
                                                                            .Prepend("").OrderBy(x => x).ToList();

    void ResetAllDropDownValues() {
        OriginCoordinates = "";
        DestinationLetter = "";
        DestinationNumber = "";
    }

    async Task<GridOptions> Visualize() {
        var items = IO.PiecesListWithEmptyCells();
        await Context.Workspace.UpdateAsync<ImportExport>(items, x => x.SnapshotMode());
        return await Context.Report.ForObjects(items.ToDataCube())
                            .GroupRowsBy(x => x.Number)
                            .GroupColumnsBy(x => x.Letter)
                            .ToTable()
                            .WithOptions(o => o.WithDefaultColumn(c => c.WithWidth(65) with { CellStyle = ChessBoardStyle, SuppressMovable = true, Resizable = false}))
                            .WithOptions(o => o with { OnGridReady = null, RowHeight = 65 })
                            .WithOptions(o => o.WithAutoGroupColumn( o => o.WithWidth(40) with { Resizable    = false,
                                                                                                 CellStyle = ChessCellStyle } ))
                            .ExecuteAsync() with {GroupHeaderHeight = 25, HeaderHeight = 0, SuppressRowHoverHighlight = true, ColumnHoverHighlight = false};
    }

    async Task<ExportResult> SaveToCsv() {
        return await Context.Export.ToCsv("SavedGame")
            .WithTable<ImportExport>(tableOptions => tableOptions.WithSource(x=> IO.PiecesList().AsQueryable()))
            .WithSource(Context.Workspace).ExecuteAsync();
    }

    async Task<ActivityLog> LoadFromCsv() {
        var acitivityLog = await Context.Import.FromFile("SavedGame.csv")
                                        .WithType<ImportExport>(x => x.SnapshotMode())
                                        .WithTarget(Context.Workspace)
                                        .ExecuteAsync();
        IO.Resume(await Context.Workspace.Query<ImportExport>().ToListAsync());
        return acitivityLog;
    }
}

In [0]:
InteractiveObject.CreateView("Application", state => state.GetScope<Application>())

In [0]:
InteractiveObject.CreateView("Game", state => state.GetScope<Application>().Visualize())

In [0]:
InteractiveObject.CreateView("Logs", state => state.GetScope<ChessBoard>().Logs)